In [222]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [223]:
train_set = pd.read_csv('./credit_train.csv', sep=';', usecols=list(range(14)), index_col='client_id')
target_set = pd.read_csv('./credit_train.csv', sep=';', usecols=[14])
s_train_set = train_set.copy(deep=True)

train_set.head()

# TODO remove to_int
def fill_empty(series, to_int=False):
    # TODO change to median?
    mean = series.mean()
    if to_int:
        mean = round(mean)
        
    return series.fillna(mean, inplace=True)

Общее количество объектов в обучающей выборке = 170746

In [192]:
len(train_set)

170746

## Исследование и преобразование фич

In [224]:
# gender. Видим, что пропусков нет. Значения пола равны 'M', 'F'. Перекодируем в числовые значения используя LabelEncoder
gender_encoder = preprocessing.LabelEncoder()
s_train_set["gender"] = gender_encoder.fit_transform(s_train_set["gender"])

# age.Всего три записи с незаполненными полями возраста. Заполним их средним по выборке значением
s_train_set["age"].fillna(round(train_set["age"].mean()), inplace=True);

# marital_status. В датасете видим незаполненные поля. Заполним данное поле самым встречаемым значением: `MAR` и перекодируем используя LabelEncoder
m_status = train_set["marital_status"]
s_train_set["marital_status"].fillna(m_status.describe()["top"], inplace=True)

marital_status_encoder = preprocessing.LabelEncoder()
s_train_set["marital_status"] = marital_status_encoder.fit_transform(s_train_set["marital_status"])

# job_position
job_encoder = preprocessing.LabelEncoder()
s_train_set["job_position"] = job_encoder.fit_transform(train_set["job_position"])

# credit_sum Преобразуем строки к float и заполним NaN средним значением по выборке
s_train_set["credit_sum"] = train_set["credit_sum"].str.replace(',', '.').astype('float')

fill_empty(s_train_set["credit_sum"])

# score_shk Возможен категориальный признак. Всего 31 значение

s_train_set["score_shk"] = s_train_set["score_shk"].str.replace(',', '.').astype('float64')
fill_empty(s_train_set["score_shk"])

# education
s_train_set["education"].fillna(s_train_set["education"].describe().top, inplace=True)
education_encoder = preprocessing.LabelEncoder()
s_train_set["education"] = education_encoder.fit_transform(s_train_set["education"])

# living_region 192 пропущенных значения
s_train_set["living_region"].fillna(s_train_set["living_region"].describe().top, inplace=True)
location_encoder = preprocessing.LabelEncoder()
s_train_set["living_region"] = education_encoder.fit_transform((s_train_set["living_region"]))

#monthly_income Было бы неплохо взять среднюю зп из людей из той же специальности и локации. Ну или близко к локации nan

fill_empty(s_train_set["monthly_income"], True)

# credit_count 9230 пропущенных значений То же самое что и с income. Надо взять всех похожих людей и засетать им те же значения
fill_empty(s_train_set["credit_count"], True)

# Overdue Credit Count 9230 пропущенных значений
# То же самое что и с income. Надо взять всех похожих людей и засетать им те же значения
# Нельзя сетать среднее значение, так как overdue_credit_count может быть больше чем credit_count
fill_empty(s_train_set["overdue_credit_count"], True)

In [225]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

s_train_set.head()

X_train, X_test, y_train, y_test = train_test_split(s_train_set, target_set, test_size=0.7)

clf = LogisticRegression(solver='lbfgs').fit(X_train, y_train);

predictions = clf.predict(X_test)

list(filter(lambda x: x == True, predictions))
#  
print(classification_report(y_test, predictions))

/home/vlad/dev/data-science-tasks/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/vlad/dev/data-science-tasks/venv/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.82      1.00      0.90     98560
           1       0.00      0.00      0.00     20963

    accuracy                           0.82    119523
   macro avg       0.41      0.50      0.45    119523
weighted avg       0.68      0.82      0.75    119523

